In [1]:
!pip install autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.1/590.1 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
 

In [ ]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

import autokeras as ak
clf = ak.ImageClassifier(max_trials=2)
clf.fit(x_train, y_train, epochs=3, verbose=2)

Trial 2 Complete [00h 27m 12s]
val_loss: 0.2689416706562042

Best val_loss So Far: 0.05385146662592888
Total elapsed time: 00h 28m 01s
Epoch 1/3
1875/1875 - 9s - loss: 0.1611 - accuracy: 0.9514 - 9s/epoch - 5ms/step
Epoch 2/3
1875/1875 - 9s - loss: 0.0734 - accuracy: 0.9773 - 9s/epoch - 5ms/step
Epoch 3/3
1875/1875 - 7s - loss: 0.0595 - accuracy: 0.9818 - 7s/epoch - 4ms/step


In [ ]:
test_loss, test_acc = clf.evaluate(x_test, y_test, verbose=0)
print("Test accuracy: ", test_acc)

Test accuracy:  0.9878000020980835


In [ ]:
predicted_y = clf.predict(x_test)
print(predicted_y)

313/313 [==============================] - 1s 4ms/step
[['7']
 ['2']
 ['1']
 ...
 ['4']
 ['5']
 ['6']]


In [ ]:
best_model = clf.export_model()
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 cast_to_float32 (CastToFlo  (None, 28, 28)            0         
 at32)                                                           
                                                                 
 expand_last_dim (ExpandLas  (None, 28, 28, 1)         0         
 tDim)                                                           
                                                                 
 normalization (Normalizati  (None, 28, 28, 1)         3         
 on)                                                             
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                             

In [ ]:
from tensorflow.keras.models import load_model

best_model.save("model_autokeras")

loaded_model = load_model("model_autokeras")  # , custom_objects=ak.CUSTOM_OBJECTS

predicted_y = loaded_model.predict(tf.expand_dims(x_test, -1))
print(predicted_y)

test_loss, test_acc = clf.evaluate(x_test, y_test, verbose=0)
print("Test accuracy: ", test_acc)

In [ ]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

clf = ak.ImageClassifier(
    max_trials=2,
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    objective="val_accuracy",
)

clf.fit(
    x_train,
    y_train,
    validation_split=0.15,
    epochs=3,
    verbose=2,
)

In [ ]:
import keras_tuner


def my_metric(y_true, y_pred):
    correct_labels = tf.cast(y_true == y_pred, tf.float32)
    return tf.reduce_mean(correct_labels, axis=-1)


clf = ak.ImageClassifier(
    seed=42,
    max_trials=2,
    loss="categorical_crossentropy",
    # Wrap the function into a Keras Tuner Objective
    # and pass it to AutoKeras.
    # Direction can be 'min' or 'max'
    # meaning we want to minimize or maximize the metric.
    # 'val_my_metric' is just add a 'val_' prefix
    # to the function name or the metric name.
    objective=keras_tuner.Objective("val_my_metric", direction="max"),
    # Include it as one of the metrics.
    metrics=[my_metric],
)

clf.fit(x_train, y_train, validation_split=0.15, epochs=3)

In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

categories = ["rec.autos", "rec.motorcycles"]

news_train = fetch_20newsgroups(
    subset="train", shuffle=True, random_state=42, categories=categories
)
news_test = fetch_20newsgroups(
    subset="test", shuffle=True, random_state=42, categories=categories
)

doc_train, label_train = np.array(news_train.data), np.array(news_train.target)
doc_test, label_test = np.array(news_test.data), np.array(news_test.target)

In [ ]:
print(
    "Unique labels {}. \nNumber of unique labels: {}.\n\n".format(
        np.unique(label_train), len(np.unique(label_train))
    )
)

print("The number of documents for training: {}.".format(len(doc_train)))
print("The number of documents for testing: {}.".format(len(doc_test)))

type(doc_train[0]), doc_train[0]

In [ ]:
# Initialize the text classifier.
clf = ak.TextClassifier(
    max_trials=2, overwrite=True
)  # It tries 3 different models. overwrite the preious history

# Feed the text classifier with training data.
clf.fit(doc_train, label_train, verbose=2)

In [ ]:
test_loss, test_acc = clf.evaluate(doc_test, label_test, verbose=0)
print("Test accuracy: ", test_acc)

In [ ]:
import tensorflow as tf

TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [ ]:
import autokeras as ak
clf = ak.ak.StructuredDataClassifier(max_trials=10)

In [ ]:
clf.fit(x=train_file_path,
        y='survived',
        verbose=2)

In [ ]:
from sklearn.datasets import fetch_california_housing
house_dataset = fetch_california_housing()

data = pd.DataFrame(house_dataset.data, columns=house_dataset.feature_names)
target = pd.Series(house_dataset.target, name="MEDV")

from sklearn.model_selection import train_test_split
train_data, test_data, train_targets, test_targets = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
regressor = ak.StructuredDataRegressor(max_trials=10, overwrite=True)
regressor.fit(x=train_data, y=train_targets, batch_size=1024)

In [ ]:
predicted_y = regressor.predict(test_data)
print(predicted_y)

In [ ]:
test_loss, test_mse = regressor.evaluate(test_data, test_targets, verbose=0)
print("Test MSE: ", test_mse)

In [ ]:
from sklearn.datasets import make_multilabel_classification

X, Y = make_multilabel_classification(
    n_samples=100,
    n_features=64,
    n_classes=3,
    n_labels=2,
    allow_unlabeled=False,
    random_state=1,
)
X = X.reshape((100, 8, 8))
X.shape, Y.shape

((100, 8, 8), (100, 3))

In [ ]:
x_train, x_test, y_train, y_test = X[:80], X[80:], Y[:80], Y[80:]

In [ ]:
# Initialize the image classifier.
clf = ak.ImageClassifier(
    max_trials=10, multi_label=True, overwrite=True
)  # It tries two different pipelines.

# Feed the image classifier with training data
# 20% of the data is used as validation data by default for tuning
# the process may run for a bit long time, please try to use GPU
clf.fit(x_train, y_train, epochs=3, verbose=2)  # each model is trained for three epochs

Trial 10 Complete [00h 00m 03s]
val_loss: 0.5362604260444641

Best val_loss So Far: 0.5287595987319946
Total elapsed time: 00h 05m 01s
Epoch 1/3
3/3 - 2s - loss: 0.6313 - accuracy: 0.4750 - 2s/epoch - 522ms/step
Epoch 2/3
3/3 - 0s - loss: 0.4554 - accuracy: 0.7875 - 27ms/epoch - 9ms/step
Epoch 3/3
3/3 - 0s - loss: 0.3957 - accuracy: 0.6750 - 26ms/epoch - 9ms/step


In [ ]:
predicted_y = clf.predict(x_test)
print("The prediction shape is: {}".format(predicted_y.shape))
print(
    "The predicted labels of the first five instances are:\n {}".format(
        predicted_y[:5, :]
    )
)

1/1 [==============================] - 0s 8ms/step
The prediction shape is: (20, 3)
The predicted labels of the first five instances are:
 [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [ ]:
test_loss, test_acc = clf.evaluate(x_test, y_test, verbose=0)
print("Test accuracy: ", test_acc)

Test accuracy:  0.6499999761581421


In [ ]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("Training image shape:", x_train.shape)  # (60000, 28, 28)
print("Training label shape:", y_train.shape)  # (60000,)
print("First five training labels:", y_train[:5])  # array([5 0 4 1 9], dtype=uint8)

Training image shape: (60000, 28, 28)
Training label shape: (60000,)
First five training labels: [5 0 4 1 9]


In [ ]:
import autokeras as ak

# Initialize the IO API.
io_model = ak.AutoModel(
    inputs=ak.ImageInput(),
    outputs=ak.ClassificationHead(
        loss="categorical_crossentropy", metrics=["accuracy"]
    ),
    objective="val_loss",
    tuner="random",
    max_trials=3,
    overwrite=True,
)

# Fit the model with prepared data.
# Use the first 100 training samples for 1 epoch as a quick demo.
# You may run with the full dataset with 10 epochs, but expect a longer training time.
io_model.fit(x_train[:100], y_train[:100], epochs=1)

Trial 3 Complete [00h 00m 07s]
val_loss: 26.515960693359375

Best val_loss So Far: 2.3032686710357666
Total elapsed time: 00h 02m 07s


4/4 [==============================] - 36s 61ms/step - loss: 2.6981 - accuracy: 0.0400


In [ ]:
io_model.tuner.results_summary()

Results summary
Results in ./auto_model
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0 summary
Hyperparameters:
image_block_1/normalize: False
image_block_1/augment: False
image_block_1/block_type: efficient
classification_head_1/spatial_reduction_1/reduction_type: global_max
classification_head_1/dropout: 0.0
optimizer: adam_weight_decay
learning_rate: 1e-05
image_block_1/efficient_net_block_1/pretrained: False
image_block_1/efficient_net_block_1/version: b0
image_block_1/efficient_net_block_1/imagenet_size: False
Score: 2.3032686710357666

Trial 1 summary
Hyperparameters:
image_block_1/normalize: False
image_block_1/augment: False
image_block_1/block_type: resnet
image_block_1/res_net_block_1/pretrained: False
image_block_1/res_net_block_1/version: resnet101_v2
image_block_1/res_net_block_1/imagenet_size: False
classification_head_1/spatial_reduction_1/reduction_type: flatten
classification_head_1/dropout: 0.5
optimizer: sgd
learning_rate: 0.0001
Score: 3

In [ ]:
best_model = io_model.export_model()
best_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28)]             0         []                            
                                                                                                  
 cast_to_float32 (CastToFlo  (None, 28, 28)               0         ['input_1[0][0]']             
 at32)                                                                                            
                                                                                                  
 expand_last_dim (ExpandLas  (None, 28, 28, 1)            0         ['cast_to_float32[0][0]']     
 tDim)                                                                                            
                                                                                              

In [ ]:
predicted_y = io_model.predict(x_test[:100])
print(predicted_y)

4/4 [==============================] - 0s 14ms/step
[['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']
 ['3']]


In [ ]:
test_loss, test_acc = io_model.evaluate(x_test[:100], y_test[:100])
print("Test accuracy: ", test_acc)

4/4 [==============================] - 2s 16ms/step - loss: 2.3027 - accuracy: 0.1100
Test accuracy:  0.10999999940395355


In [2]:
import numpy as np

num_instances = 100

# Generate image data.
image_data = np.random.rand(num_instances, 32, 32, 3).astype(np.float32)
image_train, image_test = image_data[:80], image_data[80:]

# Generate structured data.
structured_data = np.random.rand(num_instances, 20).astype(np.float32)
structured_train, structured_test = structured_data[:80], structured_data[80:]

# Generate classification labels of five classes.
classification_target = np.random.randint(5, size=num_instances)
target_train, target_test = classification_target[:80], classification_target[80:]

In [3]:
import autokeras as ak

# Initialize the IO API.
multi_input_learner = ak.AutoModel(
    inputs=[ak.ImageInput(), ak.StructuredDataInput()],
    outputs=ak.ClassificationHead(),
    max_trials=3,
    overwrite=True,
    project_name='io_api_multimodal',
)

# Fit the model with prepared data.
multi_input_learner.fit(
    [image_train, structured_train], target_train, epochs=10, verbose=2
)

Trial 3 Complete [00h 00m 44s]
val_loss: 6.279052734375

Best val_loss So Far: 1.6034681797027588
Total elapsed time: 00h 02m 38s
Epoch 1/10
3/3 - 33s - loss: 3.7430 - accuracy: 0.1000 - 33s/epoch - 11s/step
Epoch 2/10
3/3 - 0s - loss: 3.7803 - accuracy: 0.2750 - 139ms/epoch - 46ms/step
Epoch 3/10
3/3 - 0s - loss: 2.4703 - accuracy: 0.4750 - 136ms/epoch - 45ms/step
Epoch 4/10
3/3 - 0s - loss: 1.6281 - accuracy: 0.7125 - 139ms/epoch - 46ms/step
Epoch 5/10
3/3 - 0s - loss: 0.7921 - accuracy: 0.8000 - 135ms/epoch - 45ms/step
Epoch 6/10
3/3 - 0s - loss: 1.0239 - accuracy: 0.8375 - 138ms/epoch - 46ms/step
Epoch 7/10
3/3 - 0s - loss: 0.5365 - accuracy: 0.8875 - 133ms/epoch - 44ms/step
Epoch 8/10
3/3 - 0s - loss: 0.6921 - accuracy: 0.8875 - 144ms/epoch - 48ms/step
Epoch 9/10
3/3 - 0s - loss: 0.6152 - accuracy: 0.8750 - 143ms/epoch - 48ms/step
Epoch 10/10
3/3 - 0s - loss: 0.1672 - accuracy: 0.9625 - 135ms/epoch - 45ms/step


In [4]:

test_loss, test_acc = multi_input_learner.evaluate(
    [image_test, structured_test], target_test, verbose=0
)
print("Test accuracy: ", test_acc)

Test accuracy:  0.15000000596046448


In [5]:
import numpy as np

num_instances = 100

# Generate image data.
image_data = np.random.rand(num_instances, 32, 32, 3).astype(np.float32)
image_train, image_test = image_data[:80], image_data[80:]

# Generate structured data.
structured_data = np.random.rand(num_instances, 20).astype(np.float32)
structured_train, structured_test = structured_data[:80], structured_data[80:]


# Generate classification labels of five classes.
classification_target = np.random.randint(5, size=num_instances)
clf_target_train, clf_target_test = (
    classification_target[:80],
    classification_target[80:],
)

# Generate regression targets.
regression_target = np.random.rand(num_instances, 1).astype(np.float32)
reg_target_train, reg_target_test = regression_target[:80], regression_target[80:]

In [6]:

import autokeras as ak

# Initialize the IO API.
multi_output_learner = ak.AutoModel(
    inputs=[ak.ImageInput(), ak.StructuredDataInput()],
    outputs=[ak.ClassificationHead(), ak.RegressionHead()],
    max_trials=3,
    overwrite=True,
    project_name='io_api_multitask',
)

# Fit the model with prepared data.
multi_output_learner.fit(
    [image_train, structured_train],
    [clf_target_train, reg_target_train],
    epochs=10,
    verbose=2,
)

Trial 3 Complete [00h 00m 43s]
val_loss: inf

Best val_loss So Far: inf
Total elapsed time: 00h 02m 09s
Epoch 1/10
3/3 - 33s - loss: 19639.4219 - classification_head_1_loss: 115.4691 - regression_head_1_loss: 19523.9531 - classification_head_1_accuracy: 0.2125 - regression_head_1_mean_squared_error: 19523.9531 - 33s/epoch - 11s/step
Epoch 2/10
3/3 - 0s - loss: 3193.3149 - classification_head_1_loss: 39.6060 - regression_head_1_loss: 3153.7090 - classification_head_1_accuracy: 0.2375 - regression_head_1_mean_squared_error: 3153.7090 - 175ms/epoch - 58ms/step
Epoch 3/10
3/3 - 0s - loss: 676.2808 - classification_head_1_loss: 14.2580 - regression_head_1_loss: 662.0228 - classification_head_1_accuracy: 0.2875 - regression_head_1_mean_squared_error: 662.0228 - 150ms/epoch - 50ms/step
Epoch 4/10
3/3 - 0s - loss: 457.0829 - classification_head_1_loss: 10.8528 - regression_head_1_loss: 446.2301 - classification_head_1_accuracy: 0.2000 - regression_head_1_mean_squared_error: 446.2301 - 170ms/ep

In [7]:
best_model = multi_output_learner.export_model()
best_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 20)]                 0         []                            
                                                                                                  
 multi_category_encoding (M  (None, 20)                   0         ['input_2[0][0]']             
 ultiCategoryEncoding)                                                                            
                                                                                                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 dense (Dense)               (None, 32)                   672       ['multi_category_encoding[

In [9]:
import tensorflow as tf

tf.keras.utils.plot_model(best_model, show_shapes=True, expand_nested=True)

Output hidden; open in https://colab.research.google.com to view.